<div align="center">
    <h1> <b>DEPARTAMENTO DE MARKETING</b></h1>
    <h1> <b>PROYECTO DE SEGMENTACIÓN DE CLIENTES BANCARIOS</b></h1><br>
</div>
<div>
    <h3> <b>Algoritmos: clustering con K-Means y Analisis de las Componentes Principales (PCA).</b></h3>
</div>

# 1. ENUNCIADO DEL PROBLEMA Y CASO PRÁCTICO.


<table>
  <tr><td>
    <img src="https://drive.google.com/uc?id=1OjWCpwRHlCSNYaJoUUd2QGryT9CoQJ5e"
         alt="Fashion MNIST sprite"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figura 1. Segmentación de Clientes
  </td></tr>
</table>


## 1.1 Entendimiento del Negocio.

![alt text](https://drive.google.com/uc?id=1TlILFEVGZHessjmKkUMWL4NHlZ0DcHQb)

![alt text](https://drive.google.com/uc?id=1KaI1zjhARTFAs6X6VwqdsYZecd5EYDMw)

## 1.2 Objetivo del Estudio.

![alt text](https://drive.google.com/uc?id=1Q46sKROZf7jqKus0TfL7agQ5ZnzECiws)

## 1.3 Características del Cliente.

![alt text](https://drive.google.com/uc?id=1zJX7y-J2jhIL17Lh7OroOZvrlblI0u1s)

![alt text](https://drive.google.com/uc?id=1L03HfNPXpLdLlKHk_I6FIj_ahPCuPL0S)

![alt text](https://drive.google.com/uc?id=1i-OPyUph2PGuuZ5FtN0hGX3BkEXCAQNM)

Data Source: https://www.kaggle.com/arjunbhasin2013/ccdata

# 2. IMPORTAR LAS LIBRERÍAS. OBSERVACIONES PRINCIPALES DEL DATASET.

## 2.1 Importe de librerías.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, normalize # para escalar los datos
from sklearn.cluster import KMeans # para realizar la reducción de dimension
from sklearn.decomposition import PCA # analisis de componmentes principales

## 2.2 Cargar y Leer el Dataset. Configuración de la vista.

In [ ]:
# Leer el dataset
df = pd.read_csv("marketing_data.csv")
# A mi me gusta ver todas las columnas, así que realizaré una modificación en la configuración de pandas.
# Esta función recibe dos argumentos: el nombre de la opción y la cantidad de columnas que queremos visualizar. 
pd.set_option('display.max_columns',100) # lo mismo para las filas con max_rows
df.head()

## 2.3 Leyenda en español de las Columnas.

##### CUSTID: Identificación del titular de la tarjeta de crédito
##### BALANCE: Cantidad de saldo que queda en la cuenta del cliente para hacer compras
##### BALANCE_FREQUENCY: Frecuencia de la actualización del saldo, puntuación entre 0 y 1 (1 = actualizado con frecuencia, 0 = no actualizado con frecuencia)
##### PURCHASES: Cantidad de compras realizadas desde la cuenta
##### ONEOFFPURCHASES: Importe máximo de compra realizado en una sola vez
##### INSTALLMENTS_PURCHASES: Importe de la compra realizada en cuotas
##### CASH_ADVANCE: Anticipo otorgado al usuario
##### PURCHASES_FREQUENCY: frecuencia con la que se realizan las compras, puntuación entre 0 y 1 (1 = compras frecuentes, 0 = compras no frecuentes)
##### PURCHASES_FREQUENCY: Frecuencia de las Compras se están realizando, puntuación entre 0 y 1 (1 = compra con frecuencia, 0 = no compra con frecuencia)
##### ONEOFF_PURCHASES_FREQUENCY: Con qué frecuencia las compras se realizan de una sola vez (1 = compra con frecuencia, 0 = no compra con frecuencia)
##### PURCHASES_INSTALLMENTS_FREQUENCY: Con qué frecuencia se realizan las compras a plazos (1 = se realizan con frecuencia, 0 = no se realizan con frecuencia)
##### CASH_ADVANCE_FREQUENCY: con qué frecuencia el gasto se paga por adelantado
##### CASH_ADVANCE_TRX: número de transacciones realizadas con "Efectivo por adelantado"
##### PURCHASES_TRX: número de transacciones de compras realizadas
##### CREDIT_LIMIT: límite de tarjeta de crédito para el usuario
##### PAYMENTS: Número de pagos realizados por el usuario
##### MINIMUM_PAYMENTS: cantidad mínima de pagos realizados por el usuario
##### PRC_FULL_PAYMENT: porcentaje del pago total pagado por el usuario
##### TENURE: Años que el usuario lleva usando el servicio de tarjeta de crédito

## 2.4 Primeras descripciones y conclusiones.

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# La compra más alta efectuada de una sola vez es de 40761 dólares. ¿Quién hizo esta compra?
df[df['ONEOFF_PURCHASES'] == 40761.25]

In [ ]:
# El avance en efectovo más alto es de 47133 dólares. ¡Vamos a investigar quien ha hecho este avance!
df[df['CASH_ADVANCE'] == 47137.21176]

#### Al ejecutar el último código podemos observar:
#####
##### 1.- El cliente que hizo el avance más alto de todos.
##### 2.- Realizó un total de 123 transacciones por adelantado.
##### 3.- Tiene un saldo cómodo.
##### 4.- Siempre mantiene un saldo actualizado.
##### 5.- Ha realizado solo 432 vs las 49039 compras que realizó el cliente que más ha pagado por una sola compra.
##### 6.- Pidió mucho dinero, si hablamos de 47137 dólares (aprox 41,6M de pesos).
##### 7.- Reaiza compras con una frecuencia del 58%.
##### 8.- La mitad del tiempo hace uso de sus tarjetas en compras vs el cliente anterior, que usa crédito para comprar el 92% de las veces.
##### 9.- Paga siempre por adelantado vs el otro cliente que ha pagado por adelantado el 8% de las veces.
#####
##### Conclusión: podemos definir qué tipo de cliente es una persona al analizar sus datos. Por ejemplo, en este caso tenemos un cliente que siempre mantiene actualizada su tarjeta de crédito con bastante dinero. No es un cliente frecuente, pero si compra bastante, aunque solo la mitad de sus compras son realizadas con su tarjeta de crédito. Finalmente, realizó el avance más alto entre los 8950 clientes del dataset, pero se sabe que este cliente siempre paga por adelantado. ¿Qué tal? ¿Sabemos suficiente sobre este cliente?

# 3. VISUALIZACIÓN DEL DATASET

## 3.1 Transformar entradas de texto en entradas numéricas.

In [ ]:
# En este data set no hay datos de tipo texto, sin contar a CUST_ID que será eliminado. 

# Si hubiera:
# Ya que a veces algunas columnas son objetos, debemos cambiarlas a entero para poder visualizar.
# Ejemplo: el data set "jugadores"
# tiene una columna llamada "seleccionado" y las respuestas son YES and NO, entonces:
# Primero revisamos cuántas respuestas del tipo texto hay en cada columna con el código:
# jugadores['seleccionado'].value_counts()
# Luego reemplazamos la columna en cuestión por enteros usando la función lambda.
# jugadores['seleccionado'] = jugadores['seleccionado'].apply(lambda x: 1 if x == 'Yes' else 0)

## 3.2 Revisión e imputación de nulos.

### 3.2.1 Hallar nulos.

In [ ]:
df.isna().sum().sort_values(ascending = False) # se buscan, se suman y se ordenan

### 3.2.2 Mapa de calor de nulos.

In [ ]:
sns.heatmap(df.isnull(), yticklabels=False, cbar=False, cmap='Reds') # no es muy útil a mi parecer

### 3.2.3 Imputación de nulos.

##### 1.- Solo para datos numéricos se reemplazan los datos nulos directamente por la media o la media armónica según sea el caso.
##### 2.- Para datos categóricos es conveniente reemplazar por la moda si es categórico nominal u ordinal, aunque también se podría reemplazar por la media, si fueran datos categóricos ordinales. Esto se debe decidir con cuidado para no alterar demasiado los resultados.
##### 3.- También se puede eliminar la columna pero, generalmente, se perderían muchos datos importantes para el estudio.

##### Existen dos formas de hacerlo:

In [ ]:
# Forma 1: automática
df.loc[(df['MINIMUM_PAYMENTS'].isnull() == True), 'MINIMUM_PAYMENTS'] = df['MINIMUM_PAYMENTS'].mean()
df.loc[(df['CREDIT_LIMIT'].isnull() == True), 'CREDIT_LIMIT'] = df['CREDIT_LIMIT'].mean()

In [ ]:
# Forma 2: manual
# df['MINIMUM_PAYMENTS'].mean()
# df['CREDIT_LIMIT'].mean()
# Reemplazo en datos nulos de MINIMUM_PAYMENTS y CREDIT_LIMIT
#data['MINIMUM_PAYMENTS'] = data['MINIMUM_PAYMENTS'].replace(np.nan,864.2065422944161)
#data['CREDIT_LIMIT'] = data['CREDIT_LIMIT'].replace(np.nan,4494.449450364581)
# Esto no es automático

In [ ]:
# comprobar:
df.isna().sum().sort_values(ascending = False)

## 3.3 Datos duplicados.

In [ ]:
df.duplicated().sum()

## 3.4 Eliminar datos inservibles.

In [ ]:
# Podemos deshacernos del campo Customer ID porque no posee valor estadístico
df.drop(['CUST_ID'], axis=1, inplace = True)

In [ ]:
# Revisar
df.head()

## 3.5 Observación de las columnas (variables, atributos, etc).

In [ ]:
# Cantidad de columnas
cantidad_columnas = len(df.columns)
cantidad_columnas

In [ ]:
# Nombre de cada columna
df.columns

## 3.6 Graficos.

### 3.6.1 Gráfico KERNEL DENSITY ESTIMATE o gráfico de densidad de probabilidad de variable continua.

In [ ]:
# distplot combina la función matplotlib.hist con la de seaborn kdeplot()
# KDE Plot representa la Kernel Density Estimate
# KDE se utiliza para visualizar la densidad de una probabilidad de una variable continua. 
# KDE nos muestra la densidad de una probabilidad para diferentes valores de una variable continua. 

plt.figure(figsize=(10,25))
for i in range(cantidad_columnas):
    plt.subplot(cantidad_columnas,2,i+1) # filas, columnas, actual
    sns.distplot(df[df.columns[i]], kde_kws = {'color':'r', 'lw':3, 'label': 'KDE'}, hist_kws = {'color':'b'})
    plt.title(df.columns[i])

    plt.tight_layout()
    plt.legend()

### 3.6.2 Histogramas.

In [ ]:
# Ahora que tenemos la seguridad de que no hay datos nulos, realizaremos un mapa global de nuestros datos.
df.hist(bins = 30, figsize = (20, 20), color ='r')

### 3.6.3 Conclusiones.

##### 1.- La mayoría  de los clientes tiene un saldo de 1000 dólares. Este saldo se distribuye de 0 a 5000 dólares entre todos los clientes y la mayoría de ellos, lo mantienen siempre actualizado.
##### 2.- Los clientes realizan entre 0 y 3500 compras en este rango de tiempo, pero la mayoría de ellos no supera las 1000 compras. El importe máximo en una sola venta está entre 0 y 3000 dólares, pero nuevamente, la mayoría no supera los 1000 dólares.
##### 4.- Las ventas que se producen usando la tarjeta de crédito rondan entre las 0 y 1500, estando la mayoría por debajo de las 500 ventas.
##### 5.- Los avances en efectivos rondan entre 0 y 3000 dólares, estando la mayoría en torno a los 1500 dólares.
##### 6.- La frecuencia de compras y la frecuencia de compras por adelantado son bimodales, dónde la gran mayoría de usuarios no compran o compran siempre
##### 7.- Los clientes no hacen compras de grandes sumas de valor, la mayoría está entre 0 y 2%.
##### 8.- Los clientes no piden avances en efectivo, encontrándose la mayoría entre 0 y 2%.
##### 9.- La cantidad de transacciones realizadas es de 0 a 50 y las transacciones con dinero en efectivo por adelantado, entre 0 y 16.
##### 10.- El límite de las tarjetas de crédito de los clientes en su mayoría es de 1500 dólares, pero se distribuyen entre los 1000 y los 10K dólares.
##### 11.- Los pagos se distribuyen entre los 0 y los 5000 pagos aproximadamente y los pagos minimos entre 0 y 3000. Por otra parte, muy pocos clientes pagan su deuda por completo.
##### 12.- La mayoría de clientes llevan 12 años usando el servicio.

## 3.7 Correlaciones.

### 3.7.1 Matriz de Correlación.

In [ ]:
correlations = df.corr()
f, ax = plt.subplots(figsize = (12,7)) # grafico en forma de mapa de calor
sns.heatmap(correlations, annot = True) # Pintar las correlations

### 3.7.2 Correlaciones graficadas.

In [ ]:
sns.lmplot(x ='ONEOFF_PURCHASES', y ='PURCHASES', data = df, aspect = 1.5)
sns.lmplot(x='PURCHASES_INSTALLMENTS_FREQUENCY', y='PURCHASES_FREQUENCY', data = df, aspect = 1.5)
sns.lmplot(x='CASH_ADVANCE_TRX', y='CASH_ADVANCE_FREQUENCY', data = df, aspect = 1.5)

### 3.7.3 Observaciones en la Matriz.

##### 1.- ONEOFF_PURCHASES está altamente correlacionado PURCHASES
##### 2.- PURCHASES_INSTALLMENTS_FREQUENCY está altamente correlacionado con PURCHASES_FREQUENCY
##### 3.- CASH_ADVANCE_TRX está altamente correlacionado con CASH_ADVANCE_FREQUENCY

# 4. Machine Learning: K-means & PCA.

![alt text](https://drive.google.com/uc?id=1T2vAOTCInx9l2S9b9H5xm-__h-RLRY5s)

![alt text](https://drive.google.com/uc?id=1eXSerp2FQpxhAa355hKeVT0-tY94ixsw)

![alt text](https://drive.google.com/uc?id=1D12cmll79HCx_Xbt3PSq51PnytJ7DhSr)

![alt text](https://drive.google.com/uc?id=1Syuco6pZ410oP6KbfgCH18s09eB4Hd2H)

![alt text](https://drive.google.com/uc?id=1DQ4KUV66YG-xnZ4xorQFgtkx5EbE2GLI)

![alt text](https://drive.google.com/uc?id=1LzWnRt2w1bLUBGJ7tRK-b4Xudbx54H3D)

![alt text](https://drive.google.com/uc?id=1orzdGxbZPiWlLZcI7EKHAmhnz7T41cV_)

![alt text](https://drive.google.com/uc?id=1ITLmp2XTCUFIM-BPpDWOYmfjxENxywer)

# 5. Crear los data sets para posterior entrenamiento y testing. Normalización o escalamiento. Codo de Jambú.

## 5.1 Escalar el dataset.

In [ ]:
# Los números de cada variable son muy distintos entre si, por lo tanto:
scaler = StandardScaler() # escalar variables
df_scaled = scaler.fit_transform(df)

In [ ]:
df_scaled.shape

In [ ]:
df_scaled

## 5.2 Graficar el codo de jambú.

In [ ]:
wcss1 = []

for i in range(1,20):
    kmeans = KMeans(n_clusters = i, max_iter = 300)
    kmeans.fit(df_scaled) # Aquí anota que aplica K-means a la base de datos
    wcss1.append(kmeans.inertia_)

plt.plot(range(1,20),wcss1, 'bx-')
plt.title('Codo de Jambú')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS') # Es un indicador de qué tan similares son los individuos dentro de los clusters
plt.show()

### 5.3 Observaciones y Conclusión.

##### Con el gráfico podemos ver que en x = 4 es donde se forma el codo de la curva.
##### Sin embargo, según el profesor del curso, los valores no se reducen a una forma lineal hasta x = 8, por lo que él decide usar 8 clusteres.
##### Yo usaré 4.

# 6. Aplicación del algoritmo de K-means.

## 6.1 Crear el modelo de entrenamiento.

In [ ]:
kmeans = KMeans(4) # creación modelo
kmeans.fit(df_scaled) # entrenamiento del modelo
labels = kmeans.labels_ # guardamos las etiquetas que creó el modelo en labels

## 6.2 Ver los Clusters.

In [ ]:
# Ver los labels
labels

In [ ]:
# Ver cantidad de etiquetas creadas
labels.shape

In [ ]:
# Etiqueta menor
labels.min()

In [ ]:
# Etiqueta mayor
labels.max()

In [ ]:
# Ver los 8 baricentros de cada cluster
kmeans.cluster_centers_.shape # muestra en este caso las 8 observaciones x 17 valores

In [ ]:
# Ver los centros de cada cluster
cluster_centers = pd.DataFrame(data = kmeans.cluster_centers_, columns = [df.columns])      
cluster_centers # cada fila del siguiente set sería un centroide del clustering

### Observación: No es fácil entender estos valores (YA QUE SON VALORES ESCALADOS), por lo que conviene realizar la transformación inversa. ES DECIR, transformarlos nuevamente a su formato original.

## 6.3 Transformación Escalar Inverso.

In [ ]:
cluster_centers =scaler.inverse_transform(cluster_centers) # transformación escalar inversa
cluster_centers = pd.DataFrame(data = cluster_centers, columns = [df.columns]) #transformar a dataframe 
cluster_centers 

## 6.4 Predicción o Segmentación de un cliente recién llegado.

##### 1.- Ahora podemos otorgar un cluster, no solo a las 8950 personas del dataframe, si no a cualquier otro cliente que se agregue a la empresa, es decir, a otro otro dataset con nuevas observaciones. Las escalamos usando el mismo scaler y así nos podemos quedar con la nueva etiqueta que le asiganríamos a esos nuevos clientes. Esto es verdaderamente muy sencillo, porque gracias al trabajo realizado, ahora somos capaces de llevar a cabo la transformación.
##### 2.- En particular, como nuestros clientes ya los tenemos en el dataframe original, este es el momento perfecto para combinarlo todo y añadir la columna cluster al dataset, usando las labels que devuelve el algoritmo originalmente o usando el y_kmeans si queremos hacer una preddiccion final.
##### En conclusión, las labels son las mismas etiquetas que se han ido asignando a cada una de las observaciones del dataframe como parte del proceso iterativo de kmeans y las y_kmeans es lo contrario, es decir, una vez que ya tenemos el cluster definido, si yo pusiera la observacion en ese espacio de 17 dimensiones, obtendríamos cual de los 4 centros le quedaría mas cerca a ese cliente.

In [ ]:
y_kmeans = kmeans.fit_predict(df_scaled)
y_kmeans

## 6.5 Concatenar las etiquetas de clusters a la tabla original.

In [ ]:
# Concatenamos las etiquetas (labels) de los clusters con el dataset original (df)
df_cluster = pd.concat([df, pd.DataFrame({'CLUSTER':labels})], axis = 1)
df_cluster.head()

## 6.6 Definición de los grupos de clientes.

### 6.6.1 Visualización de los centroides.

In [ ]:
cluster_centers 

### 6.6.2 Frecuencia de cada Cluster.

In [ ]:
cantidad_clientes_0 = 0
cantidad_clientes_1 = 0
cantidad_clientes_2 = 0
cantidad_clientes_3 = 0
for i in df_cluster['CLUSTER']:
    if i == 0:
        cantidad_clientes_0 += 1
    elif i == 1:
        cantidad_clientes_1 += 1
    elif i == 2:
        cantidad_clientes_2 += 1
    else:
        cantidad_clientes_3 += 1    
print('Cluster 0: {} clientes. \nCluster 1: {} clientes. \nCluster 2: {} clientes. \nCluster 3: {} clientes.'.format(cantidad_clientes_0,cantidad_clientes_1,cantidad_clientes_2,cantidad_clientes_3))

### 6.6.3 Visualización de histogramas para cada cluster.

In [ ]:
# 1.- Realizaremos un gráfico para cada cluster de cada variable, es decir, 8 gráficos para cada columna.
# 2.- 
for i in df.columns:
    plt.figure(figsize = (35,5)) # grafico
    for j in range(4): # 4 gráficos para cada una de las columnas del bucle anterior
        plt.subplot(1,4,j+1) # dibujar en 1 fila, 8 columnas, donde el actual sea j+1
        cluster = df_cluster[df_cluster['CLUSTER'] == j] # me quedo con los clientes marcados en el cluster j-ésimo
        cluster[i].hist(bins = 20) # dibujamos un histograma para la columna i-ésima, con el número total de divisiones sea de 20, para compararlos graficamente entre sí
        plt.title('{}      \nCluster {}'.format(i,j)) # título para cada gráfico
    plt.show()

### 6.6.4 Definición de los grupos de clientes.

#### CLUSTER 0. NOMBRE DEL GRUPO: CLIENTES DE ALTO VOLUMEN DE AVANCES Y PROPENSOS AL NO PAGO (1198 clientes)
##### 1.- Cuentan con el saldo más alto, siendo este 4600 dolares promedio, con una concentración de 1000 a 10000 dolares y actualizado casi siempre.
##### 2.- Tienen en promedio 500 compras realizadas en su cuenta, aunque la mayoría compra muy poco. Su compra máxima no sobrepasa los 3000, con un promedio de 320 dolares.
##### 3.- Casi no usan la tarjeta de crédito, en promedio realizan 180 compras a plazo.
##### 4.- Es el grupo que pide más cantidad de dinero en avances en efectivo, en promedio 4500 dolares, con un rango de 0 a 10k dolares.
##### 5.- Son compradores poco frecuentes. La frecuencia de compra "one punch" es de un 14%. Nunca usan crédito para comprar. En promedio, piden avances de efectivo con una frecuencia del 50%.
##### 6.- Son los que realizan más transacciones con efectivo por adelantado (14). Cantidad total promedio de transacciones 7, que se mueven entre 0 y 30, con un tope de 60.
##### 7.- Límite tarjeta: 7550 dolares, tope máximo del grupo: 20k.
##### 8.- Realizan casi 3.5k pagos. Son los que realizan más pagos mínimos junto al grupo del saldo más alto. Son los que tienen menor porcentaje del pago total.
##### 9.- Aproximadamente 950 personas llevan 12 años en la empresa, el resto se divide de forma equilibrada entre los 6 y 11 años.
    

#### CLUSTER 1. NOMBRE DEL GRUPO: CLIENTES POTENCIALES PROMEDIO (3366 clientes)
##### 1.- Cuentan con el saldo más bajo, siendo este de 900 dolares promedio, más de la mitad de ellos en torno a cero, concentrado de 0 a 3000 dolares y actualizado casi siempre.
##### 2.- A pesar de ser los de menor saldo, compran dos veces más que el grupo de saldo más alto, con un rango de 0 a 5000 ventas. Son capaces de gastar entre 1000 y 2000 dolares de una vez, con un promedio de 600 dolares.
##### 3.- En promedio realizan 640 compras a plazo, pero casi la mitad de ellos realiza entre 600 y 2500 compras.
##### 4.- Es el grupo que pide menos cantidad de dinero en avances en efectivo, en promedio 210 dolares.
##### 5.- Son compradores frecuentes, un 30% de las veces compran de una vez. La mitad de ellos siempre usa crédito para comprar, los demás no siempre. Casi nunca piden anticipo de efectivo.
##### 6.- Casi no realizan transacciones con efectivo por adelantado. Cantidad total promedio de transacciones 22, que se mueven entre 0 y 37, con un tope de 110.
##### 7.- Límite tarjeta: 4200 dolares, tope máximo del grupo: 20k.
##### 8.- Realizan pocos pagos y pagos mínimos. Tienen un buen porcentaje de pago total. 
##### 9.- Casi 3000 personas llevan 12 años en la empresa (90%), el resto se divide de forma equilibrada entre los 6 y 11 años.


#### CLUSTER 2. NOMBRE DEL GRUPO: CLIENTES POTENCIALES DE ALTO VOLUMEN (409 clientes)
##### 1.- Cuentan con un saldo alto, en torno a los 3500 dolares, con una concentración de 0 a 10000 dolares y siempre actualizado.
##### 2.- Son los que más realizan compras (7600), sin embargo algunos sobrepasan las 20k, 30k, incluso las 40k ventas. Son los más capaces para realizar compras grandes de una sola vez, pueden gastar entre 1000 y 10000 dolares e incluso 15k, 25k o 40k, con un promedio de 5100 dolares.
##### 3.- En promedio, son los que más realizan compras a plazo (2600), pero existen clientes que realizan de 10k a 15k.
##### 4.- Casi nunca piden avances en efectivo. Piden sumas medias avance cuando lo hacen, en promedio 650 dolares. 
##### 5.- Son los compradores más frecuentes, con un 74%. Casi siempre usan crédito para comprar.
##### 6.- Casi no realizan transacciones con efectivo por adelantado. Cantidad total promedio de transacciones 89 (los reyes), que se mueven entre 0 y 150, con un tope de 350.
##### 7.- Tienen el límite de tarjeta más alto: 9700 dolares y un tope máximo de grupo de 30K (el más alto también).
##### 8.- Realizan más de 7k pagos. Son los que realizan más pagos mínimos junto al grupo del saldo más alto. Son los que tienen mayor porcentaje del pago total.
##### 9.- Casi todos los clientes de este grupo llevan 12 años en la empresa.


#### CLUSTER 3. NOMBRE DEL GRUPO: CLIENTES INACTIVOS Y PROPENSOS AL NO PAGO (3977 clientes)
##### 1.- Cuentan con un saldo bajo de 1000 dolares promedio, con una concentración de 0 a 3000 dolares y con la tasa más baja de los 4 grupos en actualizar su saldo.
##### 2.- Son los que menos compras tienen asociadas a su cuenta (270 compras), al menos 2550 de ellos casi no compran y el rango de venta es de 0 a 1400. Son los clientes que no realizan grandes compras de una vez (210 dolares).
##### 3.- Casi no usan la tarjeta.
##### 4.- Piden sumas medias de dinero como avance, en promedio 600 dolares.
##### 5.- Son los compradores menos frecuentes, la mitad de ellos casi no compra, la frecuencia de "one punch" es de un 9%. Nunca usan crédito para comprar. Piden muy poco anticipo de efectivo.
##### 6.- Casi no realizan transacciones con efectivo por adelantado. Cantidad total promedio de transacciones 3, que se mueven entre 0 y 8, con un tope de 15.
##### 7.- Límite tarjeta: 3280 dolares, tope máximo del grupo: 17k.
##### 8.- Son los que realizan menos pagos y pagos mínimos. Además, tienen un bajísimo niveld de porcentaje del pago total.
##### 9.- Aproximadamente 3300 personas llevan 12 años en la empresa, el resto se divide de forma equilibrada entre los 6 y 11 años.


# 7. Aplicar PCA (análisis de las componentes principales).

![alt text](https://drive.google.com/uc?id=17b_4GsvzJmWt9cnPOcgSGmEelagGSbtL)

## 7.1 Obtener las componentes principales.

In [ ]:
pca = PCA(n_components = 2)
principal_components = pca.fit_transform(df_scaled)
principal_components

## 7.2 Crear dataframe con las componentes principales.

In [ ]:
pca_df = pd.DataFrame(data = principal_components, columns = ['pca1','pca2'])
pca_df.head()

## 7.3 Concatenar las componentes principales con las etiquetas de los clusters.

In [ ]:
pca_df = pd.concat([pca_df, pd.DataFrame({'CLUSTER':labels})], axis = 1 )
pca_df.head()

## 7.4 Visualización.

### 7.4.1 Primer método.

In [ ]:
fig = plt.figure(figsize = (6,6)) #creamos una figura de 6x6

ax = fig.add_subplot(1,1,1) #indicamos que haremos solo un gráfico dentro de la figura
ax.set_xlabel('pca1', fontsize = 10) #nombres del eje x
ax.set_ylabel('pca2', fontsize = 10) #nombre del eje y
ax.set_title('Componentes Principales', fontsize = 20) #nombre del gráfico

color_theme = np.array(['blue', 'green', 'orange','red','yellow','brown','black','pink']) #creamos un array con ocho colores para cada cluster
ax.scatter(x = pca_df.pca1, y = pca_df.pca2,
          c = color_theme[pca_df.CLUSTER], s = 10)
#dibujar los puntos que representarán los vinos con scatter
#agregamos los valores de las columnas componentes
#usamos el parámetro c para indicarle los colores que deseamos para los puntos
# esta información se encuentra en KMeans_Clusters del objeto pca_nombres_vinos
# el tamaño de los puntos a graficar será de 50

plt.show()

### 7.4.2 Segundo método.

In [ ]:
# Mucho más corto que el anterior y con puntos bordeados
plt.figure(figsize = (6, 6))
ax = sns.scatterplot(x = 'pca1', y = 'pca2', hue = 'CLUSTER', data = pca_df, palette = ['blue', 'green', 'orange','red','yellow','brown','black','pink'], s = 10)
plt.show()